### RB using Fantasy Pros stats

In [1]:
%load_ext autoreload
%autoreload 2

In [47]:
import pandas as pd
pd.options.display.max_columns = None # See all columns
import warnings
warnings.filterwarnings('ignore')
from fantasy_football import Fantasy

# Import fantasy class which will be used for building datasets
football = Fantasy()

The bulk of the preprocessing logic can be found in the source code.

##### To make our dataset, lets pull running backs from the last ~10 years

In [48]:
RB_2021 = football.prepare_RB("2021")
RB_2020 = football.prepare_RB("2020")
RB_2019 = football.prepare_RB("2019")
RB_2018 = football.prepare_RB("2018")
RB_2017 = football.prepare_RB("2017")
RB_2016 = football.prepare_RB("2016")
RB_2015 = football.prepare_RB("2015")
RB_2014 = football.prepare_RB("2014")
RB_2013 = football.prepare_RB("2013")

In [49]:
all_RB = [RB_2021, RB_2020, RB_2019, RB_2018,
          RB_2017, RB_2016, RB_2015, RB_2014, RB_2013]

RB = pd.concat(all_RB, ignore_index=True)
RB

,ATT,Rushing_Yds,Y/A,LG,20+,Rushing_Td,REC,TGT,Receiving_Yds,Y/R,Receiving_Td,FPTS/G,y,YBCON,YBCON/ATT,YACON,YACON/ATT,BRKTKL,TK LOSS,TK LOSS YDS,LNG TD,10+ YDS,30+ YDS,40+ YDS,50+ YDS,RZ TGT,YACON.1
0,19.529412,106.529412,5.5,83,1.764706,1.058824,2.352941,3.000000,21.176471,9.0,0.117647,19.6,16.200000,50.882353,2.600000,55.647059,2.80000,1.470588,1.882353,-3.470588,78.000000,2.941176,0.470588,0.294118,0.176471,0.117647,5.470588
1,12.875000,56.937500,4.4,28,0.187500,0.750000,4.375000,5.875000,40.437500,9.2,0.500000,17.1,16.000000,29.750000,2.300000,27.187500,2.10000,0.812500,1.000000,-1.937500,14.000000,1.375000,0.000000,0.000000,0.000000,1.000000,10.750000
2,18.250000,75.312500,4.1,32,0.437500,0.812500,2.625000,3.000000,19.625000,7.5,0.187500,15.4,16.900000,40.937500,2.200000,34.375000,1.90000,1.250000,1.500000,-3.062500,21.000000,1.625000,0.062500,0.000000,0.000000,0.437500,3.437500
3,18.058824,70.588235,3.9,37,0.470588,0.411765,4.352941,5.529412,27.470588,6.3,0.176471,13.3,15.600000,30.882353,1.700000,39.705882,2.20000,1.764706,1.529412,-2.941176,37.000000,1.647059,0.117647,0.000000,0.000000,0.823529,12.882353
4,13.466667,50.133333,3.7,35,0.266667,1.000000,2.466667,2.600000,25.000000,10.1,0.200000,14.7,15.000000,22.066667,1.600000,28.066667,2.10000,1.266667,0.933333,-2.400000,18.000000,1.000000,0.066667,0.000000,0.000000,0.066667,9.666667
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1035,0.000000,0.000000,0.0,0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,12.621424,1.788679,15.585508,2.14717,0.000000,0.665672,-1.428860,15.622642,0.662889,0.076279,0.037310,0.019007,0.215974,0.000000
1036,0.375000,1.125000,3.0,9,0.000000,0.000000,0.500000,0.687500,6.062500,12.1,0.062500,0.0,0.000000,12.621424,1.788679,15.585508,2.14717,0.000000,0.665672,-1.428860,15.622642,0.662889,0.076279,0.037310,0.019007,0.215974,0.000000
1037,0.500000,0.928571,1.9,4,0.000000,0.000000,1.785714,2.571429,17.357143,9.7,0.000000,0.0,0.000000,12.621424,1.788679,15.585508,2.14717,0.000000,0.665672,-1.428860,15.622642,0.662889,0.076279,0.037310,0.019007,0.215974,0.000000
1038,0.000000,0.000000,0.0,0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,12.621424,1.788679,15.585508,2.14717,0.000000,0.665672,-1.428860,15.622642,0.662889,0.076279,0.037310,0.019007,0.215974,0.000000
